<a href="https://colab.research.google.com/github/SilahicAmil/NLP-NLTK/blob/main/Rotten_Tomatoes_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotten Tomatoes Sentiment

https://www.kaggle.com/mrbaloglu/rotten-tomatoes-reviews-dataset

In [1]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Tensorflow
import tensorflow as tf

#Keras
import keras
from keras.layers import Embedding, TextVectorization
from tensorflow.keras import layers

# SkLearn
from sklearn.model_selection import train_test_split

In [2]:
# Unzip data
#!unzip "/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/data_rt.csv (1).zip" -d "/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/"

In [3]:
full_data = pd.read_csv("/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/data_rt.csv")
full_data.head()

,reviews,labels
0,"simplistic , silly and tedious .",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0


In [4]:
full_data.labels.value_counts()

0    5331
1    5331
Name: labels, dtype: int64

# Shuffle Data

In [5]:
full_data = full_data.sample(frac=1, random_state=42)
full_data.head()

,reviews,labels
6830,"this film seems thirsty for reflection , itsel...",1
8600,the movie's thesis -- elegant technology for t...,1
4080,tries too hard to be funny in a way that's too...,0
3079,disturbingly superficial in its approach to th...,0
582,"an ugly , pointless , stupid movie .",0


# Creating Train/Test sets

In [6]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(full_data["reviews"].to_numpy(),
                                                                              full_data["labels"].to_numpy(),
                                                                              test_size=0.12,
                                                                              random_state=42)

In [7]:
len(train_sentences), len(test_sentences)

(9382, 1280)

In [8]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

21.04188872308676

In [9]:
MAX_VOCAB = 10_000
MAX_LEN = 21

txt_vect = TextVectorization(max_tokens=MAX_VOCAB,
                             output_mode="int",
                             output_sequence_length=MAX_LEN)

In [10]:
txt_vect.adapt(train_sentences)

# Embeddings Model - Functional API

In [11]:
Embedding = keras.layers.Embedding(input_dim=MAX_VOCAB,
                                   output_dim=128,
                                   input_length=MAX_LEN)

In [12]:
tf.random.set_seed(42)
#Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

x = txt_vect(inputs)

x = Embedding(x)

# Pooling layer
x = keras.layers.GlobalAveragePooling1D()(x)

# Dense output
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model_0 = keras.Model(inputs, outputs , name="model_0_functional")

In [13]:
model_0.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [14]:
model_0.summary()

Model: "model_0_functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 21, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,

In [15]:
hist_0 = model_0.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 6s 18ms/step - loss: 0.6615 - accuracy: 0.6489 - val_loss: 0.6054 - val_accuracy: 0.7125
Epoch 2/5
294/294 [==============================] - 5s 17ms/step - loss: 0.5131 - accuracy: 0.7848 - val_loss: 0.5140 - val_accuracy: 0.7437
Epoch 3/5
294/294 [==============================] - 5s 18ms/step - loss: 0.3822 - accuracy: 0.8567 - val_loss: 0.4956 - val_accuracy: 0.7617
Epoch 4/5
294/294 [==============================] - 4s 15ms/step - loss: 0.2981 - accuracy: 0.8897 - val_loss: 0.4987 - val_accuracy: 0.7469
Epoch 5/5
294/294 [==============================] - 3s 10ms/step - loss: 0.2392 - accuracy: 0.9188 - val_loss: 0.5171 - val_accuracy: 0.7539


# Eval

In [16]:
loss, accuracy = model_0.evaluate(test_sentences, test_labels)
print(f"Loss: {loss}\nAccuracy: {accuracy*100:2f}")

40/40 [==============================] - 0s 2ms/step - loss: 0.5171 - accuracy: 0.7539
Loss: 0.5171281099319458
Accuracy: 75.390625


# LSTM Model

In [17]:
updated_embeddings = keras.layers.Embedding(input_dim=MAX_VOCAB,
                                           output_dim=128,
                                           embeddings_initializer="uniform",
                                           input_length=MAX_LEN)

In [18]:
tf.random.set_seed(42)
# LSTM Model
inputs = keras.layers.Input(shape=(1,), dtype="string")

X = txt_vect(inputs)

X = updated_embeddings(X)

#LSTM Layers
X =keras.layers.LSTM(128)(X)
x = keras.layers.Dense(64, activation="relu")(X)

# Dense output
outputs = keras.layers.Dense(1, activation="sigmoid")(X)

model_1 = keras.Model(inputs, outputs, name="LSTM_Model")

In [19]:
model_1.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [20]:
model_1.summary()

Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
______________________________________________

In [21]:
hist_2 = model_1.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 12s 34ms/step - loss: 0.5801 - accuracy: 0.6745 - val_loss: 0.5152 - val_accuracy: 0.7484
Epoch 2/5
294/294 [==============================] - 9s 32ms/step - loss: 0.3397 - accuracy: 0.8554 - val_loss: 0.5120 - val_accuracy: 0.7523
Epoch 3/5
294/294 [==============================] - 9s 32ms/step - loss: 0.2054 - accuracy: 0.9219 - val_loss: 0.6275 - val_accuracy: 0.7500
Epoch 4/5
294/294 [==============================] - 9s 32ms/step - loss: 0.1171 - accuracy: 0.9566 - val_loss: 0.7670 - val_accuracy: 0.7469
Epoch 5/5
294/294 [==============================] - 9s 32ms/step - loss: 0.0732 - accuracy: 0.9748 - val_loss: 1.1809 - val_accuracy: 0.7266


# Eval LSTM

In [22]:
loss, accuracy = model_1.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 9ms/step - loss: 1.1809 - accuracy: 0.7266
Loss: 1.1808831691741943
Accuracy: 0.7265625


# RNN Model

In [23]:
tf.random.set_seed(42)
# Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# RNN
x = keras.layers.SimpleRNN(128)(x)

# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_2 = tf.keras.Model(inputs, outputs, name="RNN_Model")

In [24]:
model_2.summary()

Model: "RNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_______________________________________________

In [25]:
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [26]:
hist_2 = model_2.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 7s 20ms/step - loss: 0.1515 - accuracy: 0.9480 - val_loss: 0.9997 - val_accuracy: 0.7195
Epoch 2/5
294/294 [==============================] - 5s 17ms/step - loss: 0.0847 - accuracy: 0.9706 - val_loss: 0.8455 - val_accuracy: 0.7328
Epoch 3/5
294/294 [==============================] - 5s 17ms/step - loss: 0.0543 - accuracy: 0.9832 - val_loss: 1.0353 - val_accuracy: 0.7203
Epoch 4/5
294/294 [==============================] - 5s 17ms/step - loss: 0.0413 - accuracy: 0.9881 - val_loss: 1.3510 - val_accuracy: 0.7367
Epoch 5/5
294/294 [==============================] - 5s 17ms/step - loss: 0.0332 - accuracy: 0.9893 - val_loss: 1.1758 - val_accuracy: 0.7289


In [27]:
loss, accuracy = model_2.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 6ms/step - loss: 1.1758 - accuracy: 0.7289
Loss: 1.175846815109253
Accuracy: 0.7289062738418579


# GRU Model

In [31]:
tf.random.set_seed(42)

# Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

# Vect
x = txt_vect(inputs)

#Embdedd
x = updated_embeddings(x)

# GRU Cell
x = keras.layers.GRU(128)(x) # for stacking return_sequence=True is needed

# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

#Model creation
model_3 = tf.keras.Model(inputs, outputs, name="Simple_GRU_Model")

In [32]:
model_3.summary()

Model: "Simple_GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 gru_1 (GRU)                 (None, 128)               99072     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,379,201
Trainable params: 1,379,201
Non-trainable params: 0
________________________________________

In [33]:
model_3.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [34]:
hist_3 = model_3.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 18s 50ms/step - loss: 0.1321 - accuracy: 0.9491 - val_loss: 0.9369 - val_accuracy: 0.7391
Epoch 2/5
294/294 [==============================] - 14s 46ms/step - loss: 0.0382 - accuracy: 0.9884 - val_loss: 0.9602 - val_accuracy: 0.7320
Epoch 3/5
294/294 [==============================] - 8s 27ms/step - loss: 0.0214 - accuracy: 0.9931 - val_loss: 1.4300 - val_accuracy: 0.7328
Epoch 4/5
294/294 [==============================] - 8s 27ms/step - loss: 0.0166 - accuracy: 0.9952 - val_loss: 1.2345 - val_accuracy: 0.7336
Epoch 5/5
294/294 [==============================] - 8s 28ms/step - loss: 0.0146 - accuracy: 0.9962 - val_loss: 1.3817 - val_accuracy: 0.7375


In [35]:
loss, accuracy = model_3.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 8ms/step - loss: 1.3817 - accuracy: 0.7375
Loss: 1.381727695465088
Accuracy: 0.737500011920929


In [36]:
model_3_preds = model_3.predict(test_sentences)
model_3_preds[:15]

array([[2.6800334e-03],
       [2.3706257e-03],
       [9.8662877e-01],
       [3.2017350e-02],
       [2.2194088e-03],
       [1.2945235e-03],
       [9.9993944e-01],
       [7.6860189e-04],
       [9.9995422e-01],
       [9.9989343e-01],
       [4.6497583e-04],
       [9.7479135e-01],
       [9.9982262e-01],
       [2.1725267e-02],
       [9.5175326e-02]], dtype=float32)

In [37]:
# Squeezing preds so we can get a class
model_3_preds = tf.squeeze(tf.round(model_3_preds))
model_3_preds[:15]

<tf.Tensor: shape=(15,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0.],
      dtype=float32)>